<a href="https://colab.research.google.com/github/Hu8MA/Warplanes_Detection/blob/main/ProjectFinel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#2: we create new folder named(ProjectYolo7Final)

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
import os
if not os.path.isdir("ProjectYolo7Final"):
      os.makedirs("ProjectYolo7Final")


#3: access to folder "ProjectYolo"

In [ ]:
%cd ProjectYolo7Final

In [ ]:
!pwd

#4: Downlod Yolov7 in this folder & weight

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

##5: We modify the Yolo architecture through a number of steps:

A- We are working on uploading the dataSet that we have created to the Google-Drive and it is inside the path drive/Mydrive/ProjectYolo/yolov7/data

train file ,
valid file ,
test file


B- Inside Yolo/data/  
We are modifying the file COCO.yaml

My image was 6018 image , The test rate was approved 15% (903 images), where 85% remains, of which 70% is for training (4347 images) and for verification 15%
(768 images)
to like this :

train: ./data/train  # 4346 images
val: ./data/val # 768 images\n
test: ./data/test # 903 of 40670 images

number of classes
nc: 5

class names
names: ['C130', 'F15', 'F16', 'F18', 'V22']

C- Inside Yolo/cfg/training/yolov7

*   List item = 5


#6: access to yolo7 directory

In [ ]:
%cd yolov7

In [ ]:
!pwd

#**7: Now , make training for this data set**

we have tow weights **yolov7.pt** & **yolov7_training.pt**
The algorithm builder insists on using yolov7_training.pt as the starting weight, but I've tried both, and they have almost the same effect.

#Note
 this instruction was added (**--single-cls**), and it is one of the solutions that I was able to find after careful research and communication with engineers in this field, because there is a problem in the architecture when dealing with custom training data, where there is a problem in knowing the number of The entities that it will read from a file (coco) so we use this instruction to make it clear to the architecture that this data is from one entity (I know this is wrong, but our function in this project is Detection and tracking) so we do not need the classification function.

In [ ]:
!python train.py  --single-cls --device 0 --weights yolov7.pt --data data/coco.yaml  --batch-size 16 --workers 8  --epochs 55 --img 640 640  --hyp data/hyp.scratch.custom.yaml --cfg  cfg/training/yolov7.yaml  --name yolov7-custom

In [ ]:
!python train.py --single-cls  --device 0  --single-cls --weights yolov7_training.pt --data data/coco.yaml  --batch-size 16 --workers 8  --epochs 20 --img 640 640  --hyp data/hyp.scratch.custom.yaml --cfg  cfg/training/yolov7.yaml  --name yolov77-custom

#*8*:  A- Make Detectation for images

In [ ]:
!python detect.py --weights runs/train/yolov7-custom25/weights/best.pt --conf 0.25 --img-size 640  --source data/test/image

In [ ]:
!pwd

# B- **detected** on vedio


--weights "your path".pt
...... normaly , is can found in runs/train/yolov7-custom/weights/best.pt

In [ ]:
!python detect.py --weights "your path".pt --conf 0.25 --img-size 640  --source "your path".mp4

#9: Display images

In [ ]:

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('your path of one image .jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1

#if you have many images so can use
#glob.glob('your path of folder/* .jpg'):



#**Note**
If you have a video and want to test it, you must ensure that each of its frames is in the size of 640 * 640
So this code works to reset the frame size in the video so that the output is a video, all of its frames are 640 * 640

In [ ]:
import cv2

# Open the video file
video = cv2.VideoCapture('your path')

# Get the dimensions of the original video frame
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the new dimensions for the resized frame
new_width = 640
new_height = 640

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # for mp4 format
out = cv2.VideoWriter('your path to save file', fourcc, 30, (new_width, new_height))

# Loop through the video frames
while True:
    # Read a frame from the video
    ret, frame = video.read()

    # Check if we have reached the end of the video
    if not ret:
        break

    # Resize the frame
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Write the resized frame to the output video file
    out.write(resized_frame)

    # Display the resized frame
    print('Resized Frame', resized_frame)

    # Check for keyboard input to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video objects and close the window
video.release()
out.release()
cv2.destroyAllWindows()
